In [84]:
# !pip install torchtext==0.10.0
# !python -m spacy download de
# !python -m spacy download en
# !pip install -U torchtext==0.8.0

In [85]:
# import spacy
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchtext
# from torchtext.legacy.datasets import Multi30k
# from torchtext.legacy.data import Field, BucketIterator
# import spacy
# import random
# from torch import tensor

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [86]:
# # data processing block
# # tokenizer
# spacy_ger = spacy.load('de_core_news_sm')
# spacy_eng = spacy.load('en_core_web_sm')

# def ger_tokenizer(text): return [tok.text for tok in spacy_ger.tokenizer(text)]
# def eng_tokenizer(text): return [tok.text for tok in spacy_eng.tokenizer(text)]

# # field
# german = Field(tokenize=ger_tokenizer, lower=True, init_token='<sos>', eos_token='<eos>')
# english = Field(tokenize=eng_tokenizer, lower=True, init_token='<sos>', eos_token='<eos>')

# # dataset
# train_data, val_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(german, english))

# # vocabulary
# german.build_vocab(train_data, max_size=10000, min_freq=2)
# english.build_vocab(train_data, max_size=10000, min_freq=2)

# # iterator
# train_iter, val_iter, test_itr = BucketIterator.splits((train_data, val_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x: len(x.src), device=device)

In [87]:
# # encoder block
# class Encoder(nn.Module):
#   def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout_prob):
#     super().__init__()

#     self.dropout = nn.Dropout(dropout_prob)
#     self.embedding = nn.Embedding(input_size, embedding_size)
#     self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_prob, bidirectional=True)
    
#     self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size) # combines the forward and backward hidden layers for each word
#     self.fc_cell = nn.Linear(hidden_size * 2, hidden_size) # same thing for cell

#   def forward(self, x):
#     embedding = self.dropout(self.embedding(x))

#     # IMPORTANT: encoder_states contains every hidden value for every time step! (all calcuated in one pass through LSTM block because x has all the time steps in 1st dim)
#     # hidden, cell contain only the 2 context vectors (1 from forward pass and 1 from backward pass). Use small neural network to combine the two
#     # (just like when encoder_states was output, output would contain the output for every individual time step)
#     encoder_states, (hidden, cell) = self.rnn(embedding) 
#     # encoder_states: (seq_length, N, hidden_size). Need to do use fc layer to get to output size.


#     # hidden shape: (2, N, hidden_size), so want to concatenate on last axis
#     hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
#     cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

#     return encoder_states, hidden, cell


In [88]:
# # decoder block
# class Decoder(nn.Module):
#   def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout_prob):
#     super().__init__()

#     self.embedding = nn.Embedding(input_size, embedding_size)
#     self.dropout = nn.Dropout(dropout_prob)

#     # hidden_size * 2 because encoder_states is same size as hidden_size, and it is bidirectional.
#     self.rnn = nn.LSTM(hidden_size * 2 + embedding_size, hidden_size, num_layers, dropout=dropout_prob)

#     # 2 hidden_state sizes from encoder_states, 1 hidden_state size from the previous decoder output s(-1)
#     self.energy = nn.Linear(hidden_size * 3, 1)

#     self.softmax = nn.Softmax(dim=0) # why dim=0?
#     self.relu = nn.ReLU()

#     self.fc = nn.Linear(hidden_size, output_size)

#   def forward(self, x, encoder_states, hidden, cell):
#     x = x.unsqueeze(0)
#     embedded = self.dropout(self.embedding(x))

#     sequence_length = encoder_states.shape[0]
#     h_reshaped = hidden.repeat(sequence_length, 1, 1)
#     energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim=2))) # h_reshaped: hidden_size * 1, encoder_states: hidden_size * 2
#     attention = self.softmax(energy)

#     attention = attention.permute(1, 2, 0)
#     encoder_states = encoder_states.permute(1, 0, 2)

#     context_vector = torch.bmm(attention, encoder_states).permute(1, 0, 2)

#     rnn_input = torch.cat((context_vector, embedded), dim=2)

#     # attention: (seq_length, N, 1)

#     # attention = attention_weight * encoder_states
#     # how to calculate attention_weight?
    

#     output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
#     output = self.fc(output)
#     output = output.squeeze(0)
#     return output, hidden, cell

In [89]:
# # Seq2Seq block
# class Seq2Seq(nn.Module):
#   def __init__(self, encoder, decoder):
#     super().__init__()

#     self.encoder = encoder
#     self.decoder = decoder

#   def forward(self, source, target, teacher_forcing_ratio = 0.5):
#     encoder_states, hidden, cell = self.encoder(source)
#     outputs = torch.zeros((len(target), len(target[0]), len(english.vocab))).to(device)
#     x = target[0]
#     for i in range(1, len(target)):
#       output, hidden, cell = self.decoder(x, encoder_states, hidden, cell)
#       outputs[i] = output
#       best_guess = output.argmax(1)
#       x = target[i] if random.random() < teacher_forcing_ratio else best_guess
#     return outputs

In [90]:
# # train set-up block

# encoder_input_size = len(german.vocab)
# decoder_input_size = len(english.vocab)
# output_size = len(english.vocab)
# epochs = 20
# lr = 0.001
# num_layers = 1
# dropout_prob = 0.5
# embedding_size = 300
# hidden_size = 1024

# encoder = Encoder(encoder_input_size, embedding_size, hidden_size, num_layers, dropout_prob).to(device)
# decoder = Decoder(decoder_input_size, embedding_size, hidden_size, output_size, num_layers, dropout_prob).to(device)
# model = Seq2Seq(encoder, decoder).to(device)

# loss_fn = nn.CrossEntropyLoss(ignore_index=english.vocab.stoi['<pad>'])
# optimizer = optim.Adam(model.parameters(), lr=lr)

# sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

In [91]:
# # HELPER
# def translate_sentence(model, sentence, german, english, device, max_length=50):
#     spacy_ger = spacy.load("de_core_news_sm")
#     if type(sentence) == str: tokens = [token.text.lower() for token in spacy_ger(sentence)]
#     else: tokens = [token.lower() for token in sentence]
#     tokens.insert(0, german.init_token)
#     tokens.append(german.eos_token)
#     text_to_indices = [german.vocab.stoi[token] for token in tokens]
#     sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
#     with torch.no_grad(): encoder_states, hidden, cell = model.encoder(sentence_tensor)
#     outputs = [english.vocab.stoi["<sos>"]]
#     for _ in range(max_length):
#         previous_word = torch.LongTensor([outputs[-1]]).to(device)
#         with torch.no_grad():
#             output, hidden, cell = model.decoder(previous_word, encoder_states, hidden, cell)
#             best_guess = output.argmax(1).item()
#         outputs.append(best_guess)
#         if output.argmax(1).item() == english.vocab.stoi["<eos>"]: break
#     translated_sentence = [english.vocab.itos[idx] for idx in outputs]
#     return translated_sentence[1:]

In [92]:
# # training block

# for epoch in range(epochs):
#   model.eval()
#   print(f'Epoch: [{epoch} / {epochs}]')
#   training_sentence = translate_sentence(model, sentence, german, english, device)
#   print(f'Sentence: \n {training_sentence}')

#   model.train()
#   for i, batch in enumerate(train_iter):
#     xbatch = batch.src.to(device)
#     ybatch = batch.trg.to(device)

#     output = model(xbatch, ybatch)

#     output = output[1:].reshape(-1, output.shape[2])
#     ybatch = ybatch[1:].reshape(-1)

#     loss = loss_fn(output, ybatch)
#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()


In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import spacy
import random
from torch import tensor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [94]:
# data processing block

spacy_ger = spacy.load('de_core_news_sm')
spacy_eng = spacy.load('en_core_web_sm')

# tokenizers for field
def ger_tokenizer(text): return [tok.text for tok in spacy_ger.tokenizer(text)]
def eng_tokenizer(text): return [tok.text for tok in spacy_eng.tokenizer(text)]

# fields
german = Field(tokenize=ger_tokenizer, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=eng_tokenizer, lower=True, init_token='<sos>', eos_token='<eos>')

# datasets
train_data, val_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(german, english))

# vocabulary
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

# iterators
train_iter, val_itr, test_itr = BucketIterator.splits((train_data, val_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x: len(x.src), device=device)

In [95]:
from unicodedata import bidirectional
# encoding block
class Encoder(nn.Module):
  # input size: german_vocab size, embedding_size: 300, hidden_size: 1024, num_layers: 2, dropout_prob: 0.5
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout_prob):
    super().__init__()

    self.dropout = nn.Dropout(dropout_prob)
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_prob)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))

    output, (hidden, cell) = self.rnn(embedding)

    return hidden, cell

In [96]:
# decoder block
class Decoder(nn.Module):
  # input_size: english_vocab size, embedding_size: 300, hidden_size: 1024, num_layers:2, dropout_prob:0.5
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout_prob):
    super().__init__()
    self.dropout = nn.Dropout(dropout_prob)
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_prob)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden, cell):
    x = x.unsqueeze(0)
    embedding = self.dropout(self.embedding(x))
    output, (hidden, cell) = self.rnn(embedding, (hidden, cell))
    output = self.fc(output)
    output = output.squeeze(0)
    return output, hidden, cell

In [97]:
# seq2seq block
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
  
  def forward(self, source, target, teacher_force_ratio=0.5):
    hidden, cell = self.encoder(source)

    outputs = torch.zeros(len(target), len(target[0]), len(english.vocab)).to(device)
    # <sos> token
    x = target[0]
    for i in range(1, len(target)):
      output, hidden, cell = self.decoder(x, hidden, cell)
      outputs[i] = output
      best_guess = output.argmax(1)
      x = target[i] if random.random() < teacher_force_ratio else best_guess
    return outputs

In [98]:
# training set-up block
encoder_input_size = len(german.vocab)
decoder_input_size = len(english.vocab)
output_size = len(english.vocab)

embedding_size = 300
hidden_size = 1024
num_layers = 2
epochs = 20
lr = 0.001
dropout_prob = 0.5

encoder = Encoder(encoder_input_size, embedding_size, hidden_size, num_layers, dropout_prob).to(device)
decoder = Decoder(decoder_input_size, embedding_size, hidden_size, output_size, num_layers, dropout_prob).to(device)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=english.vocab.stoi['<pad>'])
optimizer = optim.Adam(model.parameters(), lr=lr)

sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

In [99]:
# HELPER
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_ger = spacy.load("de_core_news_sm")
    if type(sentence) == str: tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else: tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
    with torch.no_grad(): hidden, cell = model.encoder(sentence_tensor)
    outputs = [english.vocab.stoi["<sos>"]]
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
        outputs.append(best_guess)
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]: break
    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [100]:
# training block
for epoch in range(epochs):
  print(f'Epoch: [{epoch} / {epochs}]')
  training_sentence = translate_sentence(model, sentence, german, english, device)
  print(f'Sentence: \n {training_sentence}')

  for i, batch in enumerate(train_iter):
    xbatch = batch.src.to(device)
    ybatch = batch.trg.to(device)
    outputs = model(xbatch, ybatch)

    outputs = outputs[1:].reshape(-1, outputs.shape[2])
    ybatch = ybatch[1:].reshape(-1)

    loss = loss_fn(outputs, ybatch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


Epoch: [20 / 20]
Sentence: 
 ['latin', 'lemons', 'lemons', 'cheeks', 'inspired', 'fallen', 'pan', 'newly', 'legs', 'hides', 'pause', 'sandals', 'drive', 'drive', 'bookshelf', 'torch', 'rig', 'rig', 'rig', 'rates', 'rates', 'rates', 'breaks', 'breaks', 'arguing', 'khakis', 'environment', 'packages', 'applies', 'gazing', 'oxen', 'carve', 'tickets', 'motorbikes', 'torch', 'torch', 'home', 'facing', 'braids', 'grasping', 'coolers', 'hides', 'arrival', 'arrival', 'so', 'dolls', 'trinkets', 'fighter', 'slam', 'baked']
Epoch: [20 / 20]
Sentence: 
 ['a', 'black', 'and', 'a', 'dog', 'in', 'a', 'red', 'shirt', 'and', 'a', 'a', 'a', '.', '<eos>']
Epoch: [20 / 20]
Sentence: 
 ['a', 'snowboarder', 'with', 'a', 'is', 'is', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<eos>']
Epoch: [20 / 20]
Sentence: 
 ['a', 'worker', 'with', 'a', '<unk>', 'is', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<eos>']
Epoch: [20 / 20]
Sentence: 
 ['a', 'race', 'with', 'with', 'a', '<unk>', 'is', 'being', 'pulled', 'on', '

KeyboardInterrupt: ignored